# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 9 2022 8:30AM,245999,10,bps200075,Beach Patient Assistance,Released
1,Sep 8 2022 4:32PM,246006,19,ACG-2102491183,ACG North America LLC,Released
2,Sep 8 2022 4:32PM,246006,19,ACG-2102491184,ACG North America LLC,Released
3,Sep 8 2022 4:14PM,246004,16,SHL-8501492,"SHL Pharma, LLC",Executing
4,Sep 8 2022 4:12PM,246003,20,8475549,"ACI Healthcare USA, Inc.",Released
5,Sep 8 2022 4:12PM,246003,20,8476173,"ACI Healthcare USA, Inc.",Released
6,Sep 8 2022 4:12PM,246003,20,8476174,"ACI Healthcare USA, Inc.",Released
7,Sep 8 2022 4:12PM,246003,20,8476175,"ACI Healthcare USA, Inc.",Released
8,Sep 8 2022 4:12PM,246003,20,8476180,"ACI Healthcare USA, Inc.",Released
9,Sep 8 2022 4:12PM,246003,20,8476183,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,246001,Released,1
23,246002,Released,7
24,246003,Released,13
25,246004,Executing,1
26,246006,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
246001,NaN,NaN,NaN,1.0
246002,NaN,NaN,NaN,7.0
246003,NaN,NaN,NaN,13.0
246004,NaN,NaN,1.0,NaN
246006,NaN,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245758,1.0,0.0,0.0,0.0
245916,0.0,0.0,0.0,1.0
245917,0.0,1.0,6.0,12.0
245929,0.0,0.0,0.0,4.0
245934,0.0,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245758,1.0,0,0,0
245916,0.0,0,0,1
245917,0.0,1,6,12
245929,0.0,0,0,4
245934,0.0,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245758,1.0,0,0,0
1,245916,0.0,0,0,1
2,245917,0.0,1,6,12
3,245929,0.0,0,0,4
4,245934,0.0,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245758,1.0,,,
1,245916,0.0,,,1
2,245917,0.0,1,6,12
3,245929,0.0,,,4
4,245934,0.0,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance
1,Sep 8 2022 4:32PM,246006,19,ACG North America LLC
3,Sep 8 2022 4:14PM,246004,16,"SHL Pharma, LLC"
4,Sep 8 2022 4:12PM,246003,20,"ACI Healthcare USA, Inc."
17,Sep 8 2022 3:54PM,246002,16,Sartorius Stedim North America
24,Sep 8 2022 3:52PM,246001,16,Sartorius Stedim North America
25,Sep 8 2022 3:35PM,245998,12,Hush Hush
26,Sep 8 2022 3:34PM,245997,18,Innova Softgel LLC
27,Sep 8 2022 2:25PM,245990,10,Beach Products Inc
29,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,0.0,,,1
1,Sep 8 2022 4:32PM,246006,19,ACG North America LLC,0.0,,,2
2,Sep 8 2022 4:14PM,246004,16,"SHL Pharma, LLC",0.0,,1,
3,Sep 8 2022 4:12PM,246003,20,"ACI Healthcare USA, Inc.",0.0,,,13
4,Sep 8 2022 3:54PM,246002,16,Sartorius Stedim North America,0.0,,,7
5,Sep 8 2022 3:52PM,246001,16,Sartorius Stedim North America,0.0,,,1
6,Sep 8 2022 3:35PM,245998,12,Hush Hush,0.0,,1,
7,Sep 8 2022 3:34PM,245997,18,Innova Softgel LLC,0.0,,,1
8,Sep 8 2022 2:25PM,245990,10,Beach Products Inc,0.0,,,2
9,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1,,
1,Sep 8 2022 4:32PM,246006,19,ACG North America LLC,2,,
2,Sep 8 2022 4:14PM,246004,16,"SHL Pharma, LLC",,1,
3,Sep 8 2022 4:12PM,246003,20,"ACI Healthcare USA, Inc.",13,,
4,Sep 8 2022 3:54PM,246002,16,Sartorius Stedim North America,7,,
5,Sep 8 2022 3:52PM,246001,16,Sartorius Stedim North America,1,,
6,Sep 8 2022 3:35PM,245998,12,Hush Hush,,1,
7,Sep 8 2022 3:34PM,245997,18,Innova Softgel LLC,1,,
8,Sep 8 2022 2:25PM,245990,10,Beach Products Inc,2,,
9,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1,,
1,Sep 8 2022 4:32PM,246006,19,ACG North America LLC,2,,
2,Sep 8 2022 4:14PM,246004,16,"SHL Pharma, LLC",,1,
3,Sep 8 2022 4:12PM,246003,20,"ACI Healthcare USA, Inc.",13,,
4,Sep 8 2022 3:54PM,246002,16,Sartorius Stedim North America,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1.0,NaN,NaN
1,Sep 8 2022 4:32PM,246006,19,ACG North America LLC,2.0,NaN,NaN
2,Sep 8 2022 4:14PM,246004,16,"SHL Pharma, LLC",NaN,1.0,NaN
3,Sep 8 2022 4:12PM,246003,20,"ACI Healthcare USA, Inc.",13.0,NaN,NaN
4,Sep 8 2022 3:54PM,246002,16,Sartorius Stedim North America,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1.0,0.0,0.0
1,Sep 8 2022 4:32PM,246006,19,ACG North America LLC,2.0,0.0,0.0
2,Sep 8 2022 4:14PM,246004,16,"SHL Pharma, LLC",0.0,1.0,0.0
3,Sep 8 2022 4:12PM,246003,20,"ACI Healthcare USA, Inc.",13.0,0.0,0.0
4,Sep 8 2022 3:54PM,246002,16,Sartorius Stedim North America,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1721538,33.0,14.0,3.0
12,245998,0.0,1.0,0.0
15,491910,45.0,23.0,35.0
16,1229965,10.0,1.0,0.0
18,491931,1.0,1.0,0.0
19,491922,3.0,0.0,0.0
20,246003,13.0,0.0,0.0
22,245967,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1721538,33.0,14.0,3.0
1,12,245998,0.0,1.0,0.0
2,15,491910,45.0,23.0,35.0
3,16,1229965,10.0,1.0,0.0
4,18,491931,1.0,1.0,0.0
5,19,491922,3.0,0.0,0.0
6,20,246003,13.0,0.0,0.0
7,22,245967,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33.0,14.0,3.0
1,12,0.0,1.0,0.0
2,15,45.0,23.0,35.0
3,16,10.0,1.0,0.0
4,18,1.0,1.0,0.0
5,19,3.0,0.0,0.0
6,20,13.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,12,Released,0.0
2,15,Released,45.0
3,16,Released,10.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Completed,3.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0
Executing,14.0,1.0,23.0,1.0,1.0,0.0,0.0,0.0
Released,33.0,0.0,45.0,10.0,1.0,3.0,13.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Completed,3.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,1.0,23.0,1.0,1.0,0.0,0.0,0.0
2,Released,33.0,0.0,45.0,10.0,1.0,3.0,13.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Completed,3.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,1.0,23.0,1.0,1.0,0.0,0.0,0.0
2,Released,33.0,0.0,45.0,10.0,1.0,3.0,13.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()